Contains standard implementation of hittting set problem (unfair)

In [1]:
import sys
sys.path.append('../../..')

import numpy as np
import random
import pandas as pd
import time
from matplotlib import pyplot as plt

from core.points import *
from core.ranges import *
from algorithms.hittingset import *
from core.ranges import get_range_space
from core.verification import is_hitting_set

Rectangle:

In [8]:
def report_hittingset(n, m):
    # n = 2**10
    # m = 2**12
    # eps = 0.4
    success_prob = 0.9

    points = [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=0) for _ in range(n // 2)]
    points += [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=1) for _ in range(n // 2)]
    ranges = []
    for _ in range(m):
        xmin = random.uniform(0, 1)
        ymin = random.uniform(0, 1)
        xmax = random.uniform(xmin, 1)  # Ensure xmax > xmin
        ymax = random.uniform(ymin, 1)  # Ensure ymax > ymin
        ranges.append(RectangleRange(xmin=xmin, ymin=ymin, xmax=xmax, ymax=ymax))
    # filter out ranges with no points
    ranges = [r for r in ranges if any(r.contains(p) for p in points)]
    m = len(ranges)
    rangespace = get_range_space(points, ranges)
    vc = 2
            
    print(f"n: {n}, m: {m}, vc: {vc}")

    # for _ in range(10):
    start = time.time()
    hittingset = find_hitting_set_geometric(points, rangespace, vc)
    end = time.time()

    print(f"Number of points in epsnet: {len(hittingset)}, time taken: {end - start:.6f} seconds")

    success = is_hitting_set(hitting_set=hittingset, rangespace=rangespace)
    print(f"Success: {success}")  # Verify the eps-net
    
    return (n, m, end - start, success, 
            len([p for p in hittingset if p.color == 0]), len([p for p in hittingset if p.color == 1]))

In [10]:
n_values = [2**13, 2**14, 2**15, 2**16, 2**17]
m_values = [2**10]

aggregated_results = {
    "n": [],
    "m": [],
    "time": [],
    "success": [],
    "red_points": [],
    "blue_points": []
}

for n in n_values:
    for m in m_values:
        print(f"Running for n={n}, m={m}")
        result = report_hittingset(n, m)
        while not result[3] and tries > 0:
            print("Retrying...")
            result = report_hittingset(n, m)
            tries -= 1
        
        aggregated_results["n"].append(result[0])
        aggregated_results["m"].append(result[1])
        aggregated_results["time"].append(result[2])
        aggregated_results["success"].append(result[3])
        aggregated_results["red_points"].append(result[4])
        aggregated_results["blue_points"].append(result[5])

result = pd.DataFrame(aggregated_results)

Running for n=8192, m=1024
n: 8192, m: 1002, vc: 2
[find_hitting_set_geometric] epsilon: 0.008620689655172414
[build_epsnet_sample] epsnet size m: 5039
[find_hitting_set_geometric] epsnet size: 5039
Number of points in epsnet: 5039, time taken: 0.763424 seconds
Success: True
Running for n=16384, m=1024
n: 16384, m: 1014, vc: 2
[find_hitting_set_geometric] epsilon: 0.00827586206896552
[build_epsnet_sample] epsnet size m: 5277
[find_hitting_set_geometric] epsnet size: 5277
Number of points in epsnet: 5277, time taken: 1.760958 seconds
Success: True
Running for n=32768, m=1024
n: 32768, m: 1014, vc: 2
[find_hitting_set_geometric] epsilon: 0.008547008547008548
[build_epsnet_sample] epsnet size m: 5088
[find_hitting_set_geometric] epsnet size: 5088
Number of points in epsnet: 5088, time taken: 3.534835 seconds
Success: True
Running for n=65536, m=1024
n: 65536, m: 1021, vc: 2
[find_hitting_set_geometric] epsilon: 0.008298755186721992
[build_epsnet_sample] epsnet size m: 5261
[find_hitting_s

In [ ]:
# result.to_csv("hittingset_results_rectangle.csv", index=False)

Halfspaces

In [17]:
def report_hittingset_halfspace(n, m, dim):
    print("generating points and ranges...")
    points = [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=0) for _ in range(n // 2)]
    points += [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=1) for _ in range(n // 2)]
    ranges = []
    for _ in range(m):
        # Generate a random normal vector in R^d
        normal = [random.uniform(-1, 1) for _ in range(dim)]
        
        # Calculate the range of possible dot products with the [0, 1]^d hypercube
        min_dot = sum(min(0, n) for n in normal)  # Minimum dot product with [0, 1]^d
        max_dot = sum(max(0, n) for n in normal)  # Maximum dot product with [0, 1]^d
        
        # Choose an offset within the range [min_dot, max_dot] to ensure intersection
        offset = random.uniform(min_dot, max_dot)
        
        # Create a HalfspaceRange object
        halfspace = HalfspaceRange(normal=normal, offset=offset)
        ranges.append(halfspace)
    # filter out ranges with no points
    ranges = [r for r in ranges if any(r.contains(p) for p in points)]
    m = len(ranges)
    rangespace = get_range_space(points, ranges)
    vc = dim + 1

    print(f"n: {n}, m: {m}, vc: {vc}")
    
    # for _ in range(10):
    start = time.time()
    hittingset = find_hitting_set_geometric(points, rangespace, vc)
    end = time.time()

    print(f"Number of points in hittingset: {len(hittingset)}, time taken: {end - start:.6f} seconds")

    success = is_hitting_set(hitting_set=hittingset, rangespace=rangespace)
    print(f"Success: {success}")  # Verify the eps-net
    
    return (n, m, end - start, success, 
            len([p for p in hittingset if p.color == 0]), len([p for p in hittingset if p.color == 1]))

In [18]:
n_values = [2**13, 2**14, 2**15, 2**16, 2**17]
dims = [4, 8, 16, 32]
m_values = [2**10]

aggregated_results = {
    "n": [],
    "m": [],
    "dim": [],
    "time": [],
    "success": [],
    "red_points": [],
    "blue_points": []
}

for n in n_values:
    for m in m_values:
        for dim in dims:
            print(f"Running for n={n}, m={m}")
            result = report_hittingset_halfspace(n, m, dim)
            while not result[3] and tries > 0:
                print("Retrying...")
                result = report_hittingset_halfspace(n, m, dim)
                tries -= 1
            
            aggregated_results["n"].append(result[0])
            aggregated_results["m"].append(result[1])
            aggregated_results["time"].append(result[2])
            aggregated_results["success"].append(result[3])
            aggregated_results["red_points"].append(result[4])
            aggregated_results["blue_points"].append(result[5])
            aggregated_results["dim"].append(dim)

result = pd.DataFrame(aggregated_results)

Running for n=8192, m=1024
generating points and ranges...
n: 8192, m: 746, vc: 5
[find_hitting_set_geometric] epsilon: 0.2
[build_epsnet_sample] epsnet size m: 317
[find_hitting_set_geometric] epsnet size: 317
Number of points in hittingset: 317, time taken: 1.978666 seconds
Success: True
Running for n=8192, m=1024
generating points and ranges...
n: 8192, m: 627, vc: 9
[find_hitting_set_geometric] epsilon: 0.25
[build_epsnet_sample] epsnet size m: 432
[find_hitting_set_geometric] epsnet size: 432
Number of points in hittingset: 432, time taken: 2.608416 seconds
Success: True
Running for n=8192, m=1024
generating points and ranges...
n: 8192, m: 576, vc: 17
[find_hitting_set_geometric] epsilon: 0.25
[build_epsnet_sample] epsnet size m: 816
[find_hitting_set_geometric] epsnet size: 816
Number of points in hittingset: 816, time taken: 2.041243 seconds
Success: True
Running for n=8192, m=1024
generating points and ranges...
n: 8192, m: 537, vc: 33
[find_hitting_set_geometric] epsilon: 0.2

In [ ]:
# result.to_csv("hittingset_results_halfspace.csv", index=False)